In [9]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
REGION = 'us-central1'
BUCKET_URI = 'gs://kubeflow-mlops-410520-cloudbuild-pipelines'

In [10]:
PROJECT_ID

'kubeflow-mlops-410520'

## Create Dockerfile

In [2]:
%%writefile kfp-cli/Dockerfile


FROM gcr.io/deeplearning-platform-release/base-cpu
WORKDIR /kfp-cli
ADD pipeline.py ./pipeline.py
ADD requirements.txt ./requirements.txt
RUN pip install -r requirements.txt
CMD python ./pipeline.py


Overwriting kfp-cli/Dockerfile


## Create a bucket to store the pipeline yaml 

In [4]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://kubeflow-mlops-410520-cloudbuild-pipelines/...


## Authenticate to Artifact Registry

In [2]:
! gcloud auth configure-docker us-central1-docker.pkg.dev



{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [23]:
REPO_NAME = "mlops-starter-repo"

In [12]:
! gcloud artifacts repositories create {REPO_NAME} --repository-format=docker \
    --location={REGION} --description="Docker repository for mlops starter image" \
    --project={PROJECT_ID}

Create request issued for: [mlops-starter-repo]
Waiting for operation [projects/kubeflow-mlops-410520/locations/us-central1/ope
rations/b93785cb-e270-45e9-9400-cfbd1b64e9f4] to complete...done.              
Created repository [mlops-starter-repo].


## Build and tag the local image

In [19]:
REPO = REGION + "-docker.pkg.dev"

In [20]:
REPO

'us-central1-docker.pkg.dev'

In [27]:
REPO_NAME
IMAGE_NAME='kfp-truiz-mlops-v3'
TAG='latest'
IMAGE_URI= REPO+'/{}/{}/{}:{}'.format(PROJECT_ID,REPO_NAME, IMAGE_NAME, TAG)

In [29]:
IMAGE_URI

'us-central1-docker.pkg.dev/kubeflow-mlops-410520/mlops-starter-repo/kfp-truiz-mlops-v3:latest'

In [34]:
! docker build -t my-pipeline-image:latest ./kfp-cli

Sending build context to Docker daemon  19.97kB
Step 1/6 : FROM gcr.io/deeplearning-platform-release/base-cpu
 ---> bc9cd7f2ace0
Step 2/6 : WORKDIR /kfp-cli
 ---> Running in 7d3a851918a3
Removing intermediate container 7d3a851918a3
 ---> 8e540c39a16f
Step 3/6 : ADD pipeline.py ./pipeline.py
 ---> 5325f97747bb
Step 4/6 : ADD requirements.txt ./requirements.txt
 ---> a6175d67ca28
Step 5/6 : RUN pip install -r requirements.txt
 ---> Running in c9c25c6acde2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3

In [35]:
! docker tag my-pipeline-image:latest {IMAGE_URI}

In [ ]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} ./kfp-cli


Creating temporary tarball archive of 6 file(s) totalling 12.6 KiB before compression.
Uploading tarball of [./kfp-cli] to [gs://kubeflow-mlops-410520_cloudbuild/source/1713639360.775992-0024c203bedf4d4ab658ff9908df5e85.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/kubeflow-mlops-410520/locations/global/builds/edd77696-21ee-4f28-adc8-5786dc2e30a8].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/edd77696-21ee-4f28-adc8-5786dc2e30a8?project=990864364836 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "edd77696-21ee-4f28-adc8-5786dc2e30a8"

FETCHSOURCE
Fetching storage object: gs://kubeflow-mlops-410520_cloudbuild/source/1713639360.775992-0024c203bedf4d4ab658ff9908df5e85.tgz#1713639361019941
Copying gs://kubeflow-mlops-410520_cloudbuild/source/1713639360.775992-0024c203bedf4d4ab658ff9908df5e85.tgz#1713639361019941...
/ [1 files][  2.8 KiB/  2.8 KiB]                                                

In [36]:
! docker push {IMAGE_URI}

The push refers to repository [us-central1-docker.pkg.dev/kubeflow-mlops-410520/mlops-starter-repo/kfp-truiz-mlops-v3]

c21c2338: Preparing 
8a37977a: Preparing 
52c7e997: Preparing 
105b71e8: Preparing 
00c300cf: Preparing 
22372058: Preparing 
3ed6e28f: Preparing 
bf18a086: Preparing 
cd36d636: Preparing 
08edf074: Preparing 
3af30b89: Preparing 
c2193d47: Preparing 
0f136021: Preparing 
a9278219: Preparing 
aedb25fb: Preparing 
bd493360: Preparing 
17202fd7: Preparing 
02978576: Preparing 
30c55353: Preparing 
d211571d: Preparing 
dd7f81da: Preparing 
5bfbc4f7: Preparing 
a8a076c0: Preparing 
70758dcf: Preparing 
f18a086: Preparing 
a91e71ab: Preparing 
f18a086: Preparing 
b4b06423: Preparing 
e03c71d2: Preparing 
21c2338: Pushed lready exists MBBlatest: digest: sha256:fbcfd80b13531f70efe2bdb45492a7a6439556d9e4df6630c55d421c007a8bdf size: 7002


## Build the image using the cloudbuild.yaml file

In [39]:
!gcloud builds submit --config cloudbuild.yaml

Creating temporary tarball archive of 16 file(s) totalling 113.5 KiB before compression.
Uploading tarball of [.] to [gs://kubeflow-mlops-410520_cloudbuild/source/1713645971.724949-1339cf2a3a47422e8c464f80e2455b60.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/kubeflow-mlops-410520/locations/global/builds/c5152dcd-392c-403c-b2fe-b807b15156ca].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/c5152dcd-392c-403c-b2fe-b807b15156ca?project=990864364836 ].

gcloud builds submit only displays logs from Cloud Storage. To view logs from Cloud Logging, run:
gcloud beta builds submit

ID                                    CREATE_TIME                DURATION  SOURCE                                                                                               IMAGES  STATUS
c5152dcd-392c-403c-b2fe-b807b15156ca  2024-04-20T20:46:12+00:00  3M29S     gs://kubeflow-mlops-410520_cloudbuild/source/1713645971.724949-1339cf2a3a47422e8c464f80e2455b60.tgz  -       SUC